# Non-Graded Challenge 12
---
> Arief Luqmannurhakim
>
> Batch 001 -- FTDS


In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nelrickeine","key":"0f603eceb6345956a665633e2947a56f"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              234KB  2021-07-23 06:49:54           9662  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           5904  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2189  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2470  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [4]:
!kaggle datasets download -d hendratno/covid19-indonesia --unzip

  0% 0.00/633k [00:00<?, ?B/s]
100% 633k/633k [00:00<00:00, 40.8MB/s]


In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt


In [6]:
df = pd.read_csv('covid_19_indonesia_time_series_all.csv')
df

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,City or Regency,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,Name,Item,Kind,Hidden
0,1/8/2020,ID-JI,Jawa Timur,9,3,1,5,9,5,10,-6,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.22,0.22,0.07,0.12,55.56%,111.11%,NaN,NaN,NaN,NaN,NaN,NaN
1,1/9/2020,ID-JI,Jawa Timur,0,1,23,-24,9,6,33,-30,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.15,66.67%,366.67%,0.00,0.33,NaN,NaN,NaN,NaN
2,1/10/2020,ID-JI,Jawa Timur,0,1,14,-15,9,7,47,-45,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.17,77.78%,522.22%,1.00,1.00,NaN,NaN,NaN,NaN
3,1/11/2020,ID-JI,Jawa Timur,0,3,8,-11,9,10,55,-56,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.25,111.11%,611.11%,1.00,3.00,NaN,NaN,NaN,NaN
4,1/12/2020,ID-JI,Jawa Timur,0,3,1,-4,9,13,56,-60,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.32,144.44%,622.22%,1.00,1.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16278,7/9/2021,ID-SG,Sulawesi Tenggara,124,1,3,120,12447,252,10412,1783,Province,NaN,Sulawesi Tenggara,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,15,2.0,219,377.0,1911.0,38068,2635461,69.23,122.070311,-4.124689,47.05,4722.89,0.38,95.62,2.02%,83.65%,0.92,0.50,NaN,NaN,NaN,NaN
16279,7/9/2021,ID-SA,Sulawesi Utara,278,9,9,260,17424,572,15525,1327,Province,NaN,Sulawesi Utara,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,11,4.0,171,332.0,1507.0,13892,2641884,190.17,124.521240,1.259638,105.23,6595.29,3.41,216.51,3.28%,89.10%,1.24,9.00,NaN,NaN,NaN,NaN
16280,7/9/2021,ID-SB,Sumatera Barat,718,10,275,433,55675,1259,48840,5576,Province,NaN,Sumatera Barat,Indonesia,Asia,Sumatera,UTC+07:00,NaN,12,7.0,179,230.0,928.0,42013,5519245,131.37,100.465062,-0.850253,130.09,10087.43,1.81,228.11,2.26%,87.72%,0.93,1.00,NaN,NaN,NaN,NaN
16281,7/9/2021,ID-SS,Sumatera Selatan,341,6,221,114,31147,1569,27088,2490,Province,NaN,Sumatera Selatan,Indonesia,Asia,Sumatera,UTC+07:00,NaN,13,4.0,241,387.0,2853.0,91592,8217551,89.72,104.169465,-3.216212,41.50,3790.30,0.73,190.93,5.04%,86.97%,1.15,0.38,NaN,NaN,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16283 entries, 0 to 16282
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         16283 non-null  object 
 1   Location ISO Code            16283 non-null  object 
 2   Location                     16283 non-null  object 
 3   New Cases                    16283 non-null  int64  
 4   New Deaths                   16283 non-null  int64  
 5   New Recovered                16283 non-null  int64  
 6   New Active Cases             16283 non-null  int64  
 7   Total Cases                  16283 non-null  int64  
 8   Total Deaths                 16283 non-null  int64  
 9   Total Recovered              16283 non-null  int64  
 10  Total Active Cases           16283 non-null  int64  
 11  Location Level               16283 non-null  object 
 12  City or Regency              0 non-null      float64
 13  Province        

In [13]:
df.isna().sum()

Date                               0
Location ISO Code                  0
Location                           0
New Cases                          0
New Deaths                         0
New Recovered                      0
New Active Cases                   0
Total Cases                        0
Total Deaths                       0
Total Recovered                    0
Total Active Cases                 0
Location Level                     0
City or Regency                16283
Province                         495
Country                            0
Continent                          0
Island                           495
Time Zone                        495
Special Status                 13890
Total Regencies                    0
Total Cities                     469
Total Districts                    0
Total Urban Villages             471
Total Rural Villages             496
Area (km2)                         0
Population                         0
Population Density                 0
L

In [17]:
zero_data = df.isnull().sum() == len(df)
df.drop(columns=df.columns[zero_data], inplace=True)

In [26]:
df.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,1/8/2020,ID-JI,Jawa Timur,9,3,1,5,9,5,10,-6,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.22,0.22,0.07,0.12,55.56,111.11%,NaN,NaN
1,1/9/2020,ID-JI,Jawa Timur,0,1,23,-24,9,6,33,-30,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.15,66.67,366.67%,0.0,0.33
2,1/10/2020,ID-JI,Jawa Timur,0,1,14,-15,9,7,47,-45,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.17,77.78,522.22%,1.0,1.00
3,1/11/2020,ID-JI,Jawa Timur,0,3,8,-11,9,10,55,-56,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.25,111.11,611.11%,1.0,3.00
4,1/12/2020,ID-JI,Jawa Timur,0,3,1,-4,9,13,56,-60,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.32,144.44,622.22%,1.0,1.00


In [23]:
df.loc[:, ['Case Fatality Rate']] = df.loc[:, ['Case Fatality Rate']].applymap(lambda x: x.replace('%', '')).astype('float32').applymap(lambda x: round(x, 2))

In [32]:
conditions = [
              (df['Case Fatality Rate'] < 10),
              (df['Case Fatality Rate'] >=10) & (df['Case Fatality Rate'] < 50),
              (df['Case Fatality Rate'] > 50) & (df['Case Fatality Rate'] <= 100),
              (df['Case Fatality Rate'] > 100)
]

values = ['Warning', 'Moderate', 'Dangerous', 'Fatal']

In [33]:
df['isFatal'] = np.select(conditions, values)
df

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,isFatal
0,1/8/2020,ID-JI,Jawa Timur,9,3,1,5,9,5,10,-6,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.22,0.22,0.07,0.12,55.56,111.11%,NaN,NaN,Dangerous
1,1/9/2020,ID-JI,Jawa Timur,0,1,23,-24,9,6,33,-30,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.15,66.67,366.67%,0.00,0.33,Dangerous
2,1/10/2020,ID-JI,Jawa Timur,0,1,14,-15,9,7,47,-45,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.17,77.78,522.22%,1.00,1.00,Dangerous
3,1/11/2020,ID-JI,Jawa Timur,0,3,8,-11,9,10,55,-56,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.25,111.11,611.11%,1.00,3.00,Fatal
4,1/12/2020,ID-JI,Jawa Timur,0,3,1,-4,9,13,56,-60,Province,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.32,144.44,622.22%,1.00,1.00,Fatal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16278,7/9/2021,ID-SG,Sulawesi Tenggara,124,1,3,120,12447,252,10412,1783,Province,Sulawesi Tenggara,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,15,2.0,219,377.0,1911.0,38068,2635461,69.23,122.070311,-4.124689,47.05,4722.89,0.38,95.62,2.02,83.65%,0.92,0.50,Warning
16279,7/9/2021,ID-SA,Sulawesi Utara,278,9,9,260,17424,572,15525,1327,Province,Sulawesi Utara,Indonesia,Asia,Sulawesi,UTC+08:00,NaN,11,4.0,171,332.0,1507.0,13892,2641884,190.17,124.521240,1.259638,105.23,6595.29,3.41,216.51,3.28,89.10%,1.24,9.00,Warning
16280,7/9/2021,ID-SB,Sumatera Barat,718,10,275,433,55675,1259,48840,5576,Province,Sumatera Barat,Indonesia,Asia,Sumatera,UTC+07:00,NaN,12,7.0,179,230.0,928.0,42013,5519245,131.37,100.465062,-0.850253,130.09,10087.43,1.81,228.11,2.26,87.72%,0.93,1.00,Warning
16281,7/9/2021,ID-SS,Sumatera Selatan,341,6,221,114,31147,1569,27088,2490,Province,Sumatera Selatan,Indonesia,Asia,Sumatera,UTC+07:00,NaN,13,4.0,241,387.0,2853.0,91592,8217551,89.72,104.169465,-3.216212,41.50,3790.30,0.73,190.93,5.04,86.97%,1.15,0.38,Warning


In [34]:
observed_table = pd.crosstab(df['Location'], df['isFatal'], margins=True)
observed_table

isFatal,0,Dangerous,Fatal,Moderate,Warning,All
Location,,,,,,
Aceh,0,0,0,8,463,471
Bali,0,19,0,4,463,486
Banten,0,3,21,52,415,491
Bengkulu,6,3,0,31,426,466
DKI Jakarta,0,17,0,36,443,496
Daerah Istimewa Yogyakarta,5,2,0,21,453,481
Indonesia,0,0,0,0,495,495
Jambi,0,0,0,0,477,477
Jawa Barat,0,0,0,41,454,495


In [35]:
arr_chi = pd.crosstab(df['Location'], df['isFatal'], margins=True).values
denominator = arr_chi[-1, -1]
expected = np.zeros(tuple(np.array(arr_chi.shape) -1), dtype='float32')
for row in range(arr_chi.shape[0]-1):
  for col in range(arr_chi.shape[1] -1):

    expected[row, col] = arr_chi[row, -1] * arr_chi[-1, col] / denominator


In [36]:
index = pd.crosstab(df['Location'], df['isFatal'], margins=True).index[:-1]
column = pd.crosstab(df['Location'], df['isFatal'], margins=True).columns[:-1]
expected_table = pd.DataFrame(expected, index=index, columns=column)
expected_table = expected_table.applymap(lambda x: round(x, 3))
expected_table

isFatal,0,Dangerous,Fatal,Moderate,Warning
Location,,,,,
Aceh,0.955,2.719,4.021,21.521,441.785
Bali,0.985,2.806,4.149,22.206,455.854
Banten,0.995,2.834,4.191,22.435,460.544
Bengkulu,0.944,2.690,3.978,21.292,437.095
DKI Jakarta,1.005,2.863,4.234,22.663,465.234
Daerah Istimewa Yogyakarta,0.975,2.777,4.106,21.978,451.165
Indonesia,1.003,2.858,4.226,22.617,464.296
Jambi,0.967,2.754,4.072,21.795,447.413
Jawa Barat,1.003,2.858,4.226,22.617,464.296


In [40]:
expected_table.drop(columns=['0'], inplace=True)

In [43]:
chi2_val = ((observed_table.loc['Aceh':'Sumatera Utara', 'Dangerous':'Warning'] - expected_table)**2 / expected_table).sum().sum()
crit_val = stats.chi2.ppf(q = 0.95,  df= 99)
crit_val, chi2_val

(123.2252214533618, 2370.2429607316967)

> Chi2 value lebih dari crit_value untuk degree of freedom 99
>
> Maka --> Reject H0, varible location dan isFatal kemungkinan besar dependent, di lokasi tertentu bisa mempengaruhi case fatality rate, ini disebabkan kemungkinan karena lokasi tersebut memiliki mobilitas tinggi